In [7]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from scipy.stats import rankdata
from utils import gerar_arquiteturas

In [8]:
dados = pd.read_csv("./base_de_dados_normalizada.csv")


log_filename = 'logs.csv'


log_knn = 'log_params_knn.csv'
log_ad = 'log_params_ad.csv'
log_mlp = 'log_params_mlp.csv'
log_svm = 'log_params_svm.csv'


In [9]:
dados = shuffle(dados, random_state=1)

X = np.array(dados.iloc[:,:-1])
Y = np.array(dados.iloc[:,-1]).ravel()

# x_treino, x_temp, y_treino, y_temp = train_test_split(
#     X, Y, test_size=0.5, stratify=Y, random_state=1)
# x_validacao, x_teste, y_validacao, y_teste = train_test_split(
#     x_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=1)
X_reduzido, _, Y_reduzido, _ = train_test_split(
X, Y, test_size=0.99, stratify=Y, random_state=1)

x_treino, x_temp, y_treino, y_temp = train_test_split(
    X_reduzido, Y_reduzido, test_size=0.5, stratify=Y_reduzido, random_state=1)

x_validacao, x_teste, y_validacao, y_teste = train_test_split(
    x_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=1)



##########################################################################################################

maior = -1
for j in ("distance", "uniform"):
    for i in range(1, 51):
        KNN = KNeighborsClassifier(n_neighbors=i, weights=j)
        KNN.fit(x_treino, y_treino)
        opiniao = KNN.predict(x_validacao)
        Acc = accuracy_score(y_validacao, opiniao)
        print("K: ", i, " Métrica: ", j, " Acc: ", Acc)
        if (Acc > maior):
            maior = Acc
            Melhor_k = i
            Melhor_metrica = j


KNN = KNeighborsClassifier(n_neighbors=Melhor_k,weights=Melhor_metrica)
KNN.fit(x_treino,y_treino)
opiniao_KNN  = KNN.predict(x_teste)
prob_KNN     = KNN.predict_proba(x_teste)

accuracy_KNN = accuracy_score(y_teste, opiniao)
with open(log_knn, 'a+') as log_file:
    log_file.write(f'{1},{Melhor_k},{Melhor_metrica}\n')

###########################################################################################################

maior = -1
for j in ("entropy", "gini"):  # criterion
    for i in range(1, 11):  # max_depth
        for k in range(1, 11):  # min_samples_leaf
            for l in range(2, 16):  # min_samples_split
                for m in ('best', 'random'):  # splitter
                    AD = DecisionTreeClassifier(
                        criterion=j, max_depth=i, min_samples_leaf=k, min_samples_split=l, splitter=m)
                    AD.fit(x_treino, y_treino)
                    opiniao = AD.predict(x_validacao)
                    Acc = accuracy_score(y_validacao, opiniao)

                    if (Acc > maior):
                        maior = Acc
                        crit = j
                        md = i
                        msl = k
                        mss = l
                        split = m


AD = DecisionTreeClassifier(criterion=crit,max_depth=md,min_samples_leaf=msl,min_samples_split=mss,splitter=split)
AD.fit(x_treino,y_treino)
opiniao_AD = AD.predict(x_teste)
prob_AD = AD.predict_proba(x_teste)
accuracy_AD = accuracy_score(y_teste, opiniao)


with open(log_ad, 'a+') as log_file:
    log_file.write(f'{1},{crit},{md},{msl},{mss},{split}\n')


################################################

NB = GaussianNB()
NB.fit(x_treino,y_treino)
opiniao_NB = NB.predict(x_teste)
prob_NB = NB.predict_proba(x_teste)
accuracy_NB = accuracy_score(y_teste, opiniao_NB)

#     ################################################


best_mlp_acc = -1
best_i_mlp = 0
best_j_mlp = 0
best_k_mlp = 0
best_func_mlp = 0
best_y_pred = 0


arquiteturas = gerar_arquiteturas()

for arq in arquiteturas:
    for learning_rate in ('constant', 'invscaling', 'adaptive'):
        for epocas in (50, 100, 150, 200):
            for l in ('identity', 'logistic', 'tanh', 'relu'):

                MLP = MLPClassifier(
                    hidden_layer_sizes=arq, verbose=True, learning_rate=learning_rate, max_iter=epocas, activation=l, early_stopping=True)
                
                MLP.fit(x_treino, y_treino)

                opiniao = MLP.predict(x_validacao)
                accuracy_mlp = accuracy_score(y_validacao, opiniao)

                if (accuracy_mlp > best_mlp_acc):
                        best_mlp_acc = accuracy_mlp
                        best_i_mlp = arq
                        best_learning_rate_mlp = learning_rate
                        best_epocas_mlp = epocas
                        best_func_mlp = l

        


MLP = MLPClassifier(hidden_layer_sizes=best_i_mlp, learning_rate=best_learning_rate_mlp, max_iter=best_epocas_mlp, activation=best_func_mlp)
MLP.fit(x_treino,y_treino)
opiniao_MLP = MLP.predict(x_teste)
prob_MLP = MLP.predict_proba(x_teste)
accuracy_MLP = accuracy_score(y_teste, opiniao_MLP)
prob_opina = MLP.predict_proba(x_teste)

with open(log_mlp, 'a+') as log_file:
    log_file.write(f'{1},{best_i_mlp},{best_learning_rate_mlp},{best_epocas_mlp},{best_func_mlp}\n')


# ##########################################################################################################
## SVM


maior = -1

# Grid search nos parâmetros kernel e C
for kernel in ("linear", "poly", "rbf", "sigmoid"):
    for C in [0.1, 1, 10]: #retirei o 200
        svm = SVC(kernel=kernel, C=C, probability=True, random_state=42)
        svm.fit(x_treino, y_treino)
        opiniao = svm.predict(x_validacao)
        Acc = accuracy_score(y_validacao, opiniao)
        print(f"Kernel: {kernel}, C: {C}, Acc: {Acc:.2f}")
        if Acc > maior:
            maior = Acc
            best_kernel = kernel
            best_C = C

svm = SVC(kernel=best_kernel, C=best_C, probability=True, random_state=42)
svm.fit(x_treino, y_treino)
opiniao_SVM = svm.predict(x_teste)
prob_SVM = svm.predict_proba(x_teste)
accuracy_SVM = accuracy_score(y_teste, opiniao_SVM)
with open(log_svm, 'a+') as log_file:
    log_file.write(f'{1},{best_kernel},{best_C}\n')

K:  1  Métrica:  distance  Acc:  0.7760252365930599
K:  2  Métrica:  distance  Acc:  0.7760252365930599
K:  3  Métrica:  distance  Acc:  0.8123028391167192
K:  4  Métrica:  distance  Acc:  0.8107255520504731
K:  5  Métrica:  distance  Acc:  0.8264984227129337
K:  6  Métrica:  distance  Acc:  0.8249211356466877
K:  7  Métrica:  distance  Acc:  0.8406940063091483
K:  8  Métrica:  distance  Acc:  0.8391167192429022
K:  9  Métrica:  distance  Acc:  0.8406940063091483
K:  10  Métrica:  distance  Acc:  0.8359621451104101
K:  11  Métrica:  distance  Acc:  0.8391167192429022
K:  12  Métrica:  distance  Acc:  0.8391167192429022
K:  13  Métrica:  distance  Acc:  0.8359621451104101
K:  14  Métrica:  distance  Acc:  0.8359621451104101
K:  15  Métrica:  distance  Acc:  0.8422712933753943
K:  16  Métrica:  distance  Acc:  0.8406940063091483
K:  17  Métrica:  distance  Acc:  0.8438485804416404
K:  18  Métrica:  distance  Acc:  0.8391167192429022
K:  19  Métrica:  distance  Acc:  0.8391167192429022
K:

/home/portmap/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Kernel: linear, C: 0.1, Acc: 0.84
Kernel: linear, C: 1, Acc: 0.84
Kernel: linear, C: 10, Acc: 0.84
Kernel: poly, C: 0.1, Acc: 0.84
Kernel: poly, C: 1, Acc: 0.83
Kernel: poly, C: 10, Acc: 0.80
Kernel: rbf, C: 0.1, Acc: 0.84
Kernel: rbf, C: 1, Acc: 0.83
Kernel: rbf, C: 10, Acc: 0.82
Kernel: sigmoid, C: 0.1, Acc: 0.84
Kernel: sigmoid, C: 1, Acc: 0.81
Kernel: sigmoid, C: 10, Acc: 0.78


In [10]:
soma_prob = prob_KNN + prob_AD + prob_NB + prob_MLP + prob_SVM
opiniao_soma = np.argmax(soma_prob, axis=1)
accuracy_soma = accuracy_score(y_teste, opiniao_soma)
print(f"Accuracy Regra da Soma: {accuracy_soma}")

#voto
opinioes = np.array([opiniao_KNN, opiniao_AD, opiniao_NB, opiniao_MLP, opiniao_SVM], dtype=int) #acrescentou o dtype
opiniao_voto_majoritario = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=opinioes)
accuracy_voto_majoritario = accuracy_score(y_teste, opiniao_voto_majoritario)
print(f"Accuracy Voto Majoritário: {accuracy_voto_majoritario}")

# Borda Count - Correção
ranks = np.array([rankdata(prob_KNN[:, 1], method='min'),
                  rankdata(prob_AD[:, 1], method='min'),
                  rankdata(prob_NB[:, 1], method='min'),
                  rankdata(prob_MLP[:, 1], method='min'),
                  rankdata(prob_SVM[:, 1], method='min')])

# Soma dos ranks para cada exemplo
soma_ranks = np.sum(ranks, axis=0)

# Determinar a classe com o menor rank (melhor opção)
opiniao_borda_count = np.where(soma_ranks == np.min(soma_ranks), 1, 0)

# Calculando a acurácia
accuracy_borda_count = accuracy_score(y_teste, opiniao_borda_count)

# Log dos resultados
with open(log_filename, 'a+') as log_file:
    log_file.write(f"{'1'}, KNN, {accuracy_KNN}\n")
    log_file.write(f"{'1'}, NB, {accuracy_NB}\n")
    log_file.write(f"{'1'}, MLP, {accuracy_MLP}\n")
    log_file.write(f"{'1'}, AD, {accuracy_AD}\n")
    log_file.write(f"{'1'}, SVM, {accuracy_SVM}\n")
    log_file.write(f"{'1'}, Soma, {accuracy_soma}\n")
    log_file.write(f"{'1'}, Voto Majoritário, {accuracy_voto_majoritario}\n")
    log_file.write(f"{'1'}, Borda Count, {accuracy_borda_count}\n")

Accuracy Regra da Soma: 0.8375394321766562
Accuracy Voto Majoritário: 0.8422712933753943
